# Pull Data

In [1]:
# Imports
import pandas as pd
from sodapy import Socrata
import traceback
import os

In [2]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofnewyork.us", None)

In [3]:
DSET_ID = "9s4h-37hy"
metadata = client.get_metadata(DSET_ID)
cols = [x['name'] for x in metadata['columns']]
print(cols)

['CMPLNT_NUM', 'CMPLNT_FR_DT', 'CMPLNT_FR_TM', 'CMPLNT_TO_DT', 'CMPLNT_TO_TM', 'ADDR_PCT_CD', 'RPT_DT', 'KY_CD', 'OFNS_DESC', 'PD_CD', 'PD_DESC', 'CRM_ATPT_CPTD_CD', 'LAW_CAT_CD', 'BORO_NM', 'LOC_OF_OCCUR_DESC', 'PREM_TYP_DESC', 'JURIS_DESC', 'JURISDICTION_CODE', 'PARKS_NM', 'HADEVELOPT', 'HOUSING_PSA', 'X_COORD_CD', 'Y_COORD_CD', 'SUSP_AGE_GROUP', 'SUSP_RACE', 'SUSP_SEX', 'TRANSIT_DISTRICT', 'Latitude', 'Longitude', 'Lat_Lon (city)', 'Lat_Lon (address)', 'PATROL_BORO', 'Lat_Lon', 'STATION_NAME', 'Lat_Lon (state)', 'VIC_AGE_GROUP', 'Lat_Lon (zip)', 'VIC_RACE', 'VIC_SEX', 'Zip Codes', 'Community Districts', 'Borough Boundaries', 'City Council Districts', 'Police Precincts']


In [35]:
# returned as JSON from API / converted to Python list of dictionaries by sodapy
# loop to get all
results = pd.DataFrame()
ROWS = 10000
off = 0
while True:
    try:
        new_data = pd.read_json("https://data.cityofnewyork.us/resource/9s4h-37hy.json?$where=JURISDICTION_CODE=0&BORO_NM=MANHATTAN&$limit={}&$offset={}&$order=:id".format(ROWS, ROWS*off))
    except Exception as e:
        traceback.print_exc
        break
    results.append(new_data)
    print(len(new_data),"added")
    off+=1
# 2 column names dropped: station_name and Transit_Station
# both are almost 100% null values

50000 added


KeyboardInterrupt: 

In [ ]:
client.close()

# From Local Files

In [ ]:
# Assuming JSON files are in the same dir as this file

In [7]:
files = os.listdir('.')
files = [fname for fname in files if '.json' in fname]

In [8]:
files

['complaint_data1.json',
 'complaint_data10.json',
 'complaint_data11.json',
 'complaint_data12.json',
 'complaint_data13.json',
 'complaint_data14.json',
 'complaint_data15.json',
 'complaint_data16.json',
 'complaint_data2.json',
 'complaint_data20.json',
 'complaint_data21.json',
 'complaint_data22.json',
 'complaint_data23.json',
 'complaint_data24.json',
 'complaint_data25.json',
 'complaint_data26.json',
 'complaint_data3.json',
 'complaint_data4.json',
 'complaint_data5.json',
 'complaint_data6.json',
 'complaint_data7.json',
 'complaint_data8.json',
 'complaint_data9.json']

In [22]:
results = pd.DataFrame()
for f in files:
    results = results.append(pd.read_json(f))

In [23]:
results.head()

,addr_pct_cd,boro_nm,cmplnt_fr_dt,cmplnt_fr_tm,cmplnt_num,cmplnt_to_dt,cmplnt_to_tm,crm_atpt_cptd_cd,hadevelopt,housing_psa,...,station_name,susp_age_group,susp_race,susp_sex,transit_district,vic_age_group,vic_race,vic_sex,x_coord_cd,y_coord_cd
0,24.0,MANHATTAN,2006-04-17T00:00:00.000,17:30:00,533775188,2006-04-18T00:00:00.000,06:30:00,COMPLETED,NaN,NA,...,NaN,NaN,UNKNOWN,NaN,NaN,25-44,WHITE,M,NaN,NaN
1,9.0,MANHATTAN,2011-06-18T00:00:00.000,00:38:00,826105367,2011-06-18T00:00:00.000,00:44:00,COMPLETED,NaN,NA,...,NaN,25-44,UNKNOWN,M,NaN,25-44,WHITE HISPANIC,F,987651.0,203139.0
2,24.0,MANHATTAN,2007-09-19T00:00:00.000,08:00:00,253150967,2008-01-06T00:00:00.000,12:40:00,COMPLETED,NaN,NA,...,NaN,NaN,UNKNOWN,NaN,NaN,25-44,WHITE,M,990036.0,226881.0
3,34.0,MANHATTAN,2011-05-05T00:00:00.000,02:00:00,243272540,NaN,NaN,COMPLETED,NaN,NA,...,NaN,NaN,NaN,NaN,NaN,18-24,WHITE,F,1008127.0,256801.0
4,19.0,MANHATTAN,2006-06-08T00:00:00.000,14:25:00,186929335,2006-06-08T00:00:00.000,14:56:00,COMPLETED,NaN,NA,...,NaN,NaN,NaN,NaN,NaN,NaN,UNKNOWN,D,992738.0,218500.0


In [24]:
results.to_json('all_data.json', orient='records')

# Create dataframe for LP purposes

In [29]:
filter_cols = {'cmplnt_fr_tm': 'rj', 'cmplnt_to_tm': 'Cj','longitude':'lat','latitude': 'long', 'ofns_desc':'ofns_desc', 'law_cat_cd':'law_cat_cd'}

In [30]:
lp_df = results[list(filter_cols.keys())]
lp_df.head()

,cmplnt_fr_tm,cmplnt_to_tm,longitude,latitude,ofns_desc,law_cat_cd
0,17:30:00,06:30:00,NaN,NaN,PETIT LARCENY,MISDEMEANOR
1,00:38:00,00:44:00,-73.987731,40.724254,HARRASSMENT 2,VIOLATION
2,08:00:00,12:40:00,-73.979106,40.789418,PETIT LARCENY,MISDEMEANOR
3,02:00:00,NaN,-73.913668,40.871510,PETIT LARCENY,MISDEMEANOR
4,14:25:00,14:56:00,-73.969359,40.766413,CRIMINAL MISCHIEF & RELATED OF,MISDEMEANOR


In [34]:
lp_df.rename(filter_cols, axis=1, inplace=True)

C:\Users\Winnie\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [41]:
lp_df.to_csv("lp_dataframe.csv")

In [40]:
print(set(lp_df['ofns_desc']))

{'OFFENSES INVOLVING FRAUD', nan, 'OFFENSES AGAINST PUBLIC ADMINI', 'HOMICIDE-NEGLIGENT-VEHICLE', 'CHILD ABANDONMENT/NON SUPPORT', 'THEFT-FRAUD', 'NYS LAWS-UNCLASSIFIED VIOLATION', 'OFFENSES AGAINST THE PERSON', 'SEX CRIMES', 'ASSAULT 3 & RELATED OFFENSES', 'DANGEROUS DRUGS', 'FRAUDS', 'HARRASSMENT 2', 'OFFENSES RELATED TO CHILDREN', 'KIDNAPPING AND RELATED OFFENSES', 'BURGLARY', 'UNAUTHORIZED USE OF A VEHICLE', 'DANGEROUS WEAPONS', 'INTOXICATED & IMPAIRED DRIVING', 'HOMICIDE-NEGLIGENT,UNCLASSIFIE', 'FRAUDULENT ACCOSTING', 'CRIMINAL TRESPASS', 'THEFT OF SERVICES', 'OFFENSES AGAINST MARRIAGE UNCL', 'OTHER STATE LAWS (NON PENAL LAW)', 'OTHER OFFENSES RELATED TO THEF', 'OFFENSES AGAINST PUBLIC SAFETY', 'ARSON', 'FORGERY', 'POSSESSION OF STOLEN PROPERTY', 'PROSTITUTION & RELATED OFFENSES', 'ADMINISTRATIVE CODES', 'DISORDERLY CONDUCT', 'INTOXICATED/IMPAIRED DRIVING', 'FELONY ASSAULT', 'FORTUNE TELLING', 'ANTICIPATORY OFFENSES', 'CRIMINAL MISCHIEF & RELATED OF', 'RAPE', 'ADMINISTRATIVE CODE'

In [37]:
print(set(lp_df['law_cat_cd']))

{'MISDEMEANOR', 'FELONY', 'VIOLATION'}


In [42]:
lp_df.columns

Index(['rj', 'Cj', 'lat', 'long', 'ofns_desc', 'law_cat_cd'], dtype='object')